In [2]:
import os
import pandas as pd
import numpy as np

result_csv = '/home/tsimage/Documents/algorithm/result_12_Glandular.csv'
pics_dir = '/home/tsimage/Documents/algorithm/arrange_patches_12_Glandular/test'
#labels = ['25_ASCUS','26_LSIL','27_ASCH','28_HSIL','29_SCC']
# labels = ['20_FUNGI','21_TRI','22_CC','23_ACTINO','24_VIRUS']
labels = ['210_AGC1', '211_AGC2', '212_ADC', '213_EC']

def scan_files(directory, prefix=None, postfix=None):
    files_list = []
    for root, sub_dirs, files in os.walk(directory):
        for special_file in files:
            if postfix:
                if special_file.endswith(postfix):
                    files_list.append(os.path.join(root, special_file))
            elif prefix:
                if special_file.startswith(prefix):
                    files_list.append(os.path.join(root, special_file))
            else:
                files_list.append(os.path.join(root, special_file))
    return files_list

images = scan_files(pics_dir,postfix = '.jpg')
images = sorted(images)

def result_analysis(result_csv,images):
    df = pd.read_csv(result_csv,header=None)
    df.columns=['y','prob_0','prob_1','prob_2','prob_3']
    df['pname'] = images
    
    df = df[['pname','y','prob_0','prob_1','prob_2','prob_3']]
    df_array = np.array(df)
    df2_list = []
    for line in df_array:
        if line[1] != np.argmax(line[2:]):
            line_n = line.tolist()+[labels[int(line[1])],labels[np.argmax(line[2:])]]
            df2_list.append(line_n)
    df2_array = np.array(df2_list)
    results = pd.DataFrame(df2_array)
    results.columns = ['pname','y','210_AGC1', '211_AGC2', '212_ADC', '213_EC','y_true','y_pred']
    results.to_csv('/home/tsimage/Documents/algorithm/result_12_Glandular_ana.csv', index=False, header=True)
    
    return results
    
results = result_analysis(result_csv, images)

In [3]:
true_pred = np.array(results)[:,[-2,-1]]
true_pred_list = true_pred.tolist()
true_pred_name1 = []
true_pred_name2 = []
num = []
for t in ['210_AGC1', '211_AGC2', '212_ADC', '213_EC']:
    for p in ['210_AGC1', '211_AGC2', '212_ADC', '213_EC']:
        if t != p:
            true_pred_name1.append(t)
            true_pred_name2.append(p)
            num.append(true_pred_list.count([t,p]))

result3 = pd.DataFrame({'y_true':true_pred_name1,'y_pred':true_pred_name2,'num':num})
result3.to_csv('/home/tsimage/Documents/algorithm/result_12_Glandular_ana2.csv', index=False, header=True )